# Les variables quantitatives

## 1 Introduction

Les variables quantitatives sont les variables qui prennent des valeurs numériques. Elles sont divisées en deux catégories:

* **Une variable discrète** : Une variable est dite discrète quand elle prend ses valeurs dans uun ensemble dénombrable. Elle sert souvent à faire un comptage.
  * Exemple :
    * L'âge d'une personne
    * Le nombre d'habitant dans une ville
* **Une variable continue** : Une variable est dite continue quand elle prend ses valeurs dans un ensemble infini ou non dénombrable. Elle sert souvent à faire une mesure :
    * La taille ou le poids d'une population 
    * La distance parcourue 
    

## 2. Discrétisation d'une variable quantitative

La discrétisation consiste à transformer une variable quantitative en variable qualitative ordinale. Plus précisément, la variable quantitative est découpée en classe. Pour cela, il faut choisir le nombre de classes et les bornes de classe.

Cette technique est très importante et permet :

**Avantages :**
* Créer les liaisons non linéaires. Par exemple au lieu un coefficient pour la variable quatitative, on aura un coefficient pour chaque classe.
* Atténuer l'impact des valeurs aberrantes. Ces valeurs extrêmes seront dans des classes
* Gérer les données manquantes. On peut créer une classe correspondant aux valeurs manquantes
* ...

**Inconvénients :**
* Choisir le nombre des classes
* Choisir les bornes des classes

Il existe des formules pouvant aider à déterminer le nombre k de classes à partir du nombre n de données :

* Brooks-Carruthers  k = 5xlog<sub>10</sub>(n)
* Huntsberger        k = 1 + 3.332 x log<sub>10</sub>(n)
* Sturges            k = log<sub>2</sub>(n + 1)
* Scott              k = (max - min) / (3.5 x sigma x N <sup>-1/3</sup>)
* Freedman-Diaconis  k = (max - min) / (2 x iiq x N <sup>-1/3</sup>) 

Avec :
* sigma : écart-type des données
* iiq : intervalles interquartiles

Par défaut, les outils permettant de faire une discrétisation créent des intervalles ayant la même étendue ou des classes à effectif égal. Cependant, il existe des techniques basées sur des statistiques (écart-type, moyenne, … ) pour définir la taille des intervalles.

**Note :** La discrétisation peut ne pas avoir d'effets sur les méthodes basées sur des arbres de décision. Les arbres effectuent une discrétisation pour former les nœuds. 


In [1]:
import pandas as pd
import numpy as np

employees = [["James", 38, "Sales", 3000],
             ["Michael", 45, "Sales", 4600],
             ["Robert", 23, "Sales", 4100],
             ["Maria", 27, "Finance", 3000],
             ["James", 26, "Sales", 3000],
             ["Scott", 28, "Finance", 3300],
             ["Jen",32, "Finance", 3900],
             ["Jeff", 29, "Marketing", 3000],
             ["Kumar", 24, "Marketing", 2000],
             ["Saif", 27, "Sales", 4100]]

dfm = pd.DataFrame(employees, columns=['NAME', 'AGE', 'DEPT', 'SALARY'])

dfm

,NAME,AGE,DEPT,SALARY
0,James,38,Sales,3000
1,Michael,45,Sales,4600
2,Robert,23,Sales,4100
3,Maria,27,Finance,3000
4,James,26,Sales,3000
5,Scott,28,Finance,3300
6,Jen,32,Finance,3900
7,Jeff,29,Marketing,3000
8,Kumar,24,Marketing,2000
9,Saif,27,Sales,4100


**Discrétiser en fixant les intervalles (étendue des classes)**

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import FunctionTransformer
bins = [20, 25, 30, 35, np.inf]
labels = ['Group 1', 'Group 2', 'Group 3', 'Group 4']

transformer = FunctionTransformer(pd.cut, 
                                  kw_args={'bins': bins, 
                                           'labels': labels,
                                           'retbins': False}
                                 )

_ = transformer.fit(dfm['AGE'])
dfm['AGE_BINNED'] = transformer.transform(dfm['AGE'])

dfm

,NAME,AGE,DEPT,SALARY,AGE_BINNED
0,James,38,Sales,3000,Group 4
1,Michael,45,Sales,4600,Group 4
2,Robert,23,Sales,4100,Group 1
3,Maria,27,Finance,3000,Group 2
4,James,26,Sales,3000,Group 2
5,Scott,28,Finance,3300,Group 2
6,Jen,32,Finance,3900,Group 3
7,Jeff,29,Marketing,3000,Group 2
8,Kumar,24,Marketing,2000,Group 1
9,Saif,27,Sales,4100,Group 2


**Discritiser à l'aide des quantiles**

In [3]:
from sklearn.preprocessing import KBinsDiscretizer

model = KBinsDiscretizer(n_bins=4, encode='ordinal', strategy='quantile')

_ = model.fit(dfm[['AGE']])
dfm['AGE_BINNED_QUANTILE'] = model.transform(dfm[['AGE']])

dfm

,NAME,AGE,DEPT,SALARY,AGE_BINNED,AGE_BINNED_QUANTILE
0,James,38,Sales,3000,Group 4,3.0
1,Michael,45,Sales,4600,Group 4,3.0
2,Robert,23,Sales,4100,Group 1,0.0
3,Maria,27,Finance,3000,Group 2,1.0
4,James,26,Sales,3000,Group 2,0.0
5,Scott,28,Finance,3300,Group 2,2.0
6,Jen,32,Finance,3900,Group 3,3.0
7,Jeff,29,Marketing,3000,Group 2,2.0
8,Kumar,24,Marketing,2000,Group 1,0.0
9,Saif,27,Sales,4100,Group 2,1.0


## 3. Transformations des variables quantitatives


### 3.1  Transformation non lineaire

Certaines méthodes statistiques supposent une normalité des variables (suivent une loi normale). Dans la pratique, cette condition n'est pas souvent respectée. Des transformations peuvent être appliquées aux données pour avoir des données qui suivent une distribution symétrique. En fonction de la caractéristique des données plus plusieurs types de transformations :

* **Transformation logarithme :** stabiliser la variance et normaliser les distributions avec une asymétrie positive
* **Transformation racine carrée :** transformer des données asymétriques
* **Transformation inverse :**   rapprocher les valeurs extrêmes des valeurs moyennes 

Transformation logarithme

In [4]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log10, validate=True)
_ = transformer.fit(dfm[['SALARY']])
dfm['LOG_SALARY'] = transformer.transform(dfm[['SALARY']])

dfm

,NAME,AGE,DEPT,SALARY,AGE_BINNED,AGE_BINNED_QUANTILE,LOG_SALARY
0,James,38,Sales,3000,Group 4,3.0,3.477121
1,Michael,45,Sales,4600,Group 4,3.0,3.662758
2,Robert,23,Sales,4100,Group 1,0.0,3.612784
3,Maria,27,Finance,3000,Group 2,1.0,3.477121
4,James,26,Sales,3000,Group 2,0.0,3.477121
5,Scott,28,Finance,3300,Group 2,2.0,3.518514
6,Jen,32,Finance,3900,Group 3,3.0,3.591065
7,Jeff,29,Marketing,3000,Group 2,2.0,3.477121
8,Kumar,24,Marketing,2000,Group 1,0.0,3.301030
9,Saif,27,Sales,4100,Group 2,1.0,3.612784


Transformation racine carré

In [5]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.sqrt, validate=True)

_ = transformer.fit(dfm[['SALARY']])
dfm['SQRT_SALARY'] = transformer.transform(dfm[['SALARY']])

dfm

,NAME,AGE,DEPT,SALARY,AGE_BINNED,AGE_BINNED_QUANTILE,LOG_SALARY,SQRT_SALARY
0,James,38,Sales,3000,Group 4,3.0,3.477121,54.772256
1,Michael,45,Sales,4600,Group 4,3.0,3.662758,67.823300
2,Robert,23,Sales,4100,Group 1,0.0,3.612784,64.031242
3,Maria,27,Finance,3000,Group 2,1.0,3.477121,54.772256
4,James,26,Sales,3000,Group 2,0.0,3.477121,54.772256
5,Scott,28,Finance,3300,Group 2,2.0,3.518514,57.445626
6,Jen,32,Finance,3900,Group 3,3.0,3.591065,62.449980
7,Jeff,29,Marketing,3000,Group 2,2.0,3.477121,54.772256
8,Kumar,24,Marketing,2000,Group 1,0.0,3.301030,44.721360
9,Saif,27,Sales,4100,Group 2,1.0,3.612784,64.031242


### 3.2 Feature scaling

Ce sont des techniques qui ajustent les données pour rendre comparables :

* **StandardScaler (Standardisation) :** centrer et réduire une variable en retranchant pour chaque vecteur sa moyenne et en divisant par son écart-type.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(dfm[['AGE', 'SALARY']])
dfm_scaled = scaler.transform(dfm[['AGE', 'SALARY']])
dfm_scaled = pd.DataFrame(dfm_scaled, columns=['AGE_SCALED', 'SALARY_SCALED'], index=dfm.index)
dfm.join(dfm_scaled)

,NAME,AGE,DEPT,SALARY,AGE_BINNED,AGE_BINNED_QUANTILE,LOG_SALARY,SQRT_SALARY,AGE_SCALED,SALARY_SCALED
0,James,38,Sales,3000,Group 4,3.0,3.477121,54.772256,1.254495,-0.550482
1,Michael,45,Sales,4600,Group 4,3.0,3.662758,67.823300,2.338627,1.651446
2,Robert,23,Sales,4100,Group 1,0.0,3.612784,64.031242,-1.068644,0.963343
3,Maria,27,Finance,3000,Group 2,1.0,3.477121,54.772256,-0.449140,-0.550482
4,James,26,Sales,3000,Group 2,0.0,3.477121,54.772256,-0.604016,-0.550482
5,Scott,28,Finance,3300,Group 2,2.0,3.518514,57.445626,-0.294264,-0.137620
6,Jen,32,Finance,3900,Group 3,3.0,3.591065,62.449980,0.325240,0.688102
7,Jeff,29,Marketing,3000,Group 2,2.0,3.477121,54.772256,-0.139388,-0.550482
8,Kumar,24,Marketing,2000,Group 1,0.0,3.301030,44.721360,-0.913768,-1.926687
9,Saif,27,Sales,4100,Group 2,1.0,3.612784,64.031242,-0.449140,0.963343


* **MinMaxScaler :** ajuster une variable de sorte qu'elle prennnent ses valeurs dans un ensemble bien défini

In [7]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(dfm[['AGE', 'SALARY']])
dfm_scaled = scaler.transform(dfm[['AGE', 'SALARY']])
dfm_scaled = pd.DataFrame(dfm_scaled, columns=['AGE_SCALED', 'SALARY_SCALED'], index=dfm.index)

dfm.join(dfm_scaled)

,NAME,AGE,DEPT,SALARY,AGE_BINNED,AGE_BINNED_QUANTILE,LOG_SALARY,SQRT_SALARY,AGE_SCALED,SALARY_SCALED
0,James,38,Sales,3000,Group 4,3.0,3.477121,54.772256,0.681818,0.384615
1,Michael,45,Sales,4600,Group 4,3.0,3.662758,67.823300,1.000000,1.000000
2,Robert,23,Sales,4100,Group 1,0.0,3.612784,64.031242,0.000000,0.807692
3,Maria,27,Finance,3000,Group 2,1.0,3.477121,54.772256,0.181818,0.384615
4,James,26,Sales,3000,Group 2,0.0,3.477121,54.772256,0.136364,0.384615
5,Scott,28,Finance,3300,Group 2,2.0,3.518514,57.445626,0.227273,0.500000
6,Jen,32,Finance,3900,Group 3,3.0,3.591065,62.449980,0.409091,0.730769
7,Jeff,29,Marketing,3000,Group 2,2.0,3.477121,54.772256,0.272727,0.384615
8,Kumar,24,Marketing,2000,Group 1,0.0,3.301030,44.721360,0.045455,0.000000
9,Saif,27,Sales,4100,Group 2,1.0,3.612784,64.031242,0.181818,0.807692


* **RobustScaler :** identique à Min-Max mais utilise l’intervalle interquartile au lieu des valeurs Min et Max.

In [8]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler().fit(dfm[['AGE', 'SALARY']])
dfm_scaled = scaler.transform(dfm[['AGE', 'SALARY']])
dfm_scaled = pd.DataFrame(dfm_scaled, columns=['AGE_SCALED', 'SALARY_SCALED'], index=dfm.index)

dfm.join(dfm_scaled)

,NAME,AGE,DEPT,SALARY,AGE_BINNED,AGE_BINNED_QUANTILE,LOG_SALARY,SQRT_SALARY,AGE_SCALED,SALARY_SCALED
0,James,38,Sales,3000,Group 4,3.0,3.477121,54.772256,2.1,-0.142857
1,Michael,45,Sales,4600,Group 4,3.0,3.662758,67.823300,3.5,1.380952
2,Robert,23,Sales,4100,Group 1,0.0,3.612784,64.031242,-0.9,0.904762
3,Maria,27,Finance,3000,Group 2,1.0,3.477121,54.772256,-0.1,-0.142857
4,James,26,Sales,3000,Group 2,0.0,3.477121,54.772256,-0.3,-0.142857
5,Scott,28,Finance,3300,Group 2,2.0,3.518514,57.445626,0.1,0.142857
6,Jen,32,Finance,3900,Group 3,3.0,3.591065,62.449980,0.9,0.714286
7,Jeff,29,Marketing,3000,Group 2,2.0,3.477121,54.772256,0.3,-0.142857
8,Kumar,24,Marketing,2000,Group 1,0.0,3.301030,44.721360,-0.7,-1.095238
9,Saif,27,Sales,4100,Group 2,1.0,3.612784,64.031242,-0.1,0.904762


## 4. Gestion des données manquantes

Quelques stratégies pour gérer les données manquantes (DM) :

* Certaines méthodes comme Xgboost tolèrent les DM
* Suppression des DM lorsque les conditions le permettent 
* Imputation avec une statistique de variable (moyenne, médiane, etc, ...)
* Imputation avec méthode des plus proches voisins (KNN)
* Imputation avec une régression locale
* Imputation via une SVD (décomposition en valeurs singulières)

In [9]:
import pandas as pd
import numpy as np

employees = [["James", 38, "Sales", 3000],
             ["Michael", None, "Sales", 4600],
             ["Robert", 23, "Sales", 4100],
             ["Maria", 27, "Finance", 3000],
             ["James", 26, "Sales", 3000],
             ["Scott", np.NAN, "Finance", 3300],
             ["Jen",32, "Finance", 3900],
             ["Jeff", 29, "Marketing", 3000],
             ["Kumar", np.NaN, "Marketing", 2000],
             ["Saif", 27, "Sales", 4100]]

dfm = pd.DataFrame(employees, columns=['NAME', 'AGE', 'DEPT', 'SALARY'])

dfm

,NAME,AGE,DEPT,SALARY
0,James,38.0,Sales,3000
1,Michael,NaN,Sales,4600
2,Robert,23.0,Sales,4100
3,Maria,27.0,Finance,3000
4,James,26.0,Sales,3000
5,Scott,NaN,Finance,3300
6,Jen,32.0,Finance,3900
7,Jeff,29.0,Marketing,3000
8,Kumar,NaN,Marketing,2000
9,Saif,27.0,Sales,4100


Imputation par la moyenne

In [10]:
import numpy as np
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(missing_values=np.nan, strategy='mean').fit(dfm[['AGE']])

_ = imputer.fit(dfm[['AGE']])
dfm['AGE_IMPUTED'] = imputer.transform(dfm[['AGE']])

dfm

,NAME,AGE,DEPT,SALARY,AGE_IMPUTED
0,James,38.0,Sales,3000,38.000000
1,Michael,NaN,Sales,4600,28.857143
2,Robert,23.0,Sales,4100,23.000000
3,Maria,27.0,Finance,3000,27.000000
4,James,26.0,Sales,3000,26.000000
5,Scott,NaN,Finance,3300,28.857143
6,Jen,32.0,Finance,3900,32.000000
7,Jeff,29.0,Marketing,3000,29.000000
8,Kumar,NaN,Marketing,2000,28.857143
9,Saif,27.0,Sales,4100,27.000000


Imputation par la médiane 

In [11]:
imputer = SimpleImputer(missing_values=np.nan, strategy='median').fit(dfm[['AGE']])

_ = imputer.fit(dfm[['AGE']])
dfm['AGE_IMPUTED'] = imputer.transform(dfm[['AGE']])

dfm

,NAME,AGE,DEPT,SALARY,AGE_IMPUTED
0,James,38.0,Sales,3000,38.0
1,Michael,NaN,Sales,4600,27.0
2,Robert,23.0,Sales,4100,23.0
3,Maria,27.0,Finance,3000,27.0
4,James,26.0,Sales,3000,26.0
5,Scott,NaN,Finance,3300,27.0
6,Jen,32.0,Finance,3900,32.0
7,Jeff,29.0,Marketing,3000,29.0
8,Kumar,NaN,Marketing,2000,27.0
9,Saif,27.0,Sales,4100,27.0


Imputation par une constante (par 30 exemple)

In [12]:
imputer = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=30)

dfm['AGE_IMPUTED'] = imputer.fit_transform(dfm[['AGE']])

dfm

,NAME,AGE,DEPT,SALARY,AGE_IMPUTED
0,James,38.0,Sales,3000,38.0
1,Michael,NaN,Sales,4600,30.0
2,Robert,23.0,Sales,4100,23.0
3,Maria,27.0,Finance,3000,27.0
4,James,26.0,Sales,3000,26.0
5,Scott,NaN,Finance,3300,30.0
6,Jen,32.0,Finance,3900,32.0
7,Jeff,29.0,Marketing,3000,29.0
8,Kumar,NaN,Marketing,2000,30.0
9,Saif,27.0,Sales,4100,27.0


Imputation par knn

In [13]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(missing_values=np.nan, n_neighbors=3)

dfm['AGE_IMPUTED'] = imputer.fit_transform(dfm[['AGE']])

dfm

,NAME,AGE,DEPT,SALARY,AGE_IMPUTED
0,James,38.0,Sales,3000,38.000000
1,Michael,NaN,Sales,4600,28.857143
2,Robert,23.0,Sales,4100,23.000000
3,Maria,27.0,Finance,3000,27.000000
4,James,26.0,Sales,3000,26.000000
5,Scott,NaN,Finance,3300,28.857143
6,Jen,32.0,Finance,3900,32.000000
7,Jeff,29.0,Marketing,3000,29.000000
8,Kumar,NaN,Marketing,2000,28.857143
9,Saif,27.0,Sales,4100,27.000000


[Documentation de Scikit-learn](https://scikit-learn.org/stable/)